In [8]:
!pip install selenium webdriver-manager beautifulsoup4 pandas requests lxml

In [9]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

options = Options()
options.add_argument("--headless=new")  # chế độ ẩn cửa sổ
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)


In [10]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import pandas as pd
import requests
import time
import random
from datetime import datetime

In [18]:
# ===================== CẤU HÌNH =====================
SITES = {
    "vietnamworks": "https://www.vietnamworks.com/jobs",
    "careerviet": "https://careerviet.vn/viec-lam",
    "topcv": "https://www.topcv.vn/viec-lam",
}
MAX_LOAD_MORE = 5   # số lần scroll/load thêm trang
CRAWL_DATE = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

# Thêm header tránh bị chặn khi requests nội dung chi tiết
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) "
                  "Chrome/120.0 Safari/537.36"
}

# ===================================================

In [19]:
# --- Setup Chrome (headless, không cần driver path) ---
options = Options()
options.add_argument("--headless=new")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--disable-blink-features=AutomationControlled")

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
wait = WebDriverWait(driver, 10)

all_jobs = []

# ===================================================

In [20]:
# --- Crawl từng trang ---
# ===================================================

for site_name, start_url in SITES.items():
    print(f"\n💼 Đang crawl trang: {site_name}")
    driver.get(start_url)
    time.sleep(2)

    # --- Scroll để load thêm việc ---
    for i in range(MAX_LOAD_MORE):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(random.uniform(2, 3))

    # --- Lấy toàn bộ HTML ---
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")

    # ===================================================
    # Tùy chỉnh CSS selector theo từng trang cụ thể
    # ===================================================
    job_blocks = []
    if site_name == "vietnamworks":
        job_blocks = soup.select("li.job-item")
        # job_blocks = soup.select("div.job-list__item") or soup.select("div.job-item")
        print(soup.prettify()[:1000])
        print(len(soup.select("div.job-list__item")))
    elif site_name == "careerbuilder":
        job_blocks = soup.select("div.job-item") or soup.select(".job-element")
        print(soup.prettify()[:1000])
        print(len(soup.select("div.job-item")))
    elif site_name == "topcv":
        job_blocks = soup.select("div.job-item-search-result")
        print(soup.prettify()[:1000])
        print(len(soup.select("div.job-item-search-result")))

    print(f"🔍 Tìm thấy {len(job_blocks)} việc làm trên {site_name}")

    # ===================================================
    # Parse thông tin từ từng khối job
    # ===================================================
    for job in job_blocks:
        try:
            title_elem = job.select_one("a[href]")
            if not title_elem:
                continue

            title = title_elem.get_text(strip=True)
            link = title_elem.get("href")
            if not link.startswith("http"):
                link = "https://" + site_name + ".vn" + link

            company = None
            location = None
            salary = None
            posted_time = None

            # Tùy biến cho từng trang
            if site_name == "vietnamworks":
                company = job.select_one(".company-name")
                location = job.select_one(".job-location")
                salary = job.select_one(".job-salary")
                posted_time = job.select_one(".posted-date")

            elif site_name == "careerviet":
                company = job.select_one(".company a")
                location = job.select_one(".location")
                salary = job.select_one(".salary")
                posted_time = job.select_one(".time")

            elif site_name == "topcv":
                company = job.select_one(".company-name a")
                location = job.select_one(".address")
                salary = job.select_one(".salary")
                posted_time = job.select_one(".time")

            job_data = {
                "source": site_name,
                "title": title,
                "company": company.get_text(strip=True) if company else None,
                "location": location.get_text(strip=True) if location else None,
                "salary": salary.get_text(strip=True) if salary else None,
                "time_posted": posted_time.get_text(strip=True) if posted_time else None,
                "link": link,
                "crawl_date": CRAWL_DATE
            }
            all_jobs.append(job_data)
        except Exception as e:
            print("⚠️ Lỗi parse job:", e)

print(f"\n✅ Tổng số việc thu được: {len(all_jobs)}")
driver.quit()



💼 Đang crawl trang: vietnamworks
<html>
 <head>
  <title>
   406 Not Acceptable
  </title>
 </head>
 <body>
  <center>
   <h1>
    406 Not Acceptable
   </h1>
  </center>
  <hr/>
  <center>
   nginx
  </center>
 </body>
</html>

0
🔍 Tìm thấy 0 việc làm trên vietnamworks

💼 Đang crawl trang: careerviet
🔍 Tìm thấy 0 việc làm trên careerviet

💼 Đang crawl trang: topcv
<html lang="vi">
 <head>
  <meta charset="utf-8"/>
  <meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
  <meta content="width=device-width, initial-scale=1.0, maximum-scale=1" name="viewport"/>
  <link href="https://static.topcv.vn/v4/image/favicon.ico" rel="icon" type="image/x-icon"/>
  <meta content="1478418029113221" property="fb:app_id"/>
  <meta content="1kv5ZE3gPUfgx42i3AGLLN2ys54dSKyvx246kBa3" name="csrf-token"/>
  <meta content="okX9WYRRKSr_EuYuN-C58UPMnAAusti-V-UFcpJEVZg" name="google-site-verification"/>
  <meta content="UTd2SHpqb1hYdC9mUnV4YUFDN3U0QT090" name="dmca-site-verification"/>
  <link href=

In [6]:
# ===================================================
# --- Lấy nội dung chi tiết từng job (nếu có) ---
# ===================================================
print("\n🧠 Đang tải nội dung chi tiết từng job...")
for job in all_jobs:
    try:
        html = requests.get(job["link"], headers=HEADERS, timeout=10).text
        soup = BeautifulSoup(html, "html.parser")
        content = " ".join([p.get_text(strip=True) for p in soup.select("div.job-description, div#job-detail, div.job-info-content p")])
        job["content"] = content[:1000]  # chỉ lấy tối đa 1000 ký tự mô tả
        time.sleep(random.uniform(0.5, 1.2))
    except Exception as e:
        print(f"Lỗi tải chi tiết: {e}")
        job["content"] = ""

# ===================================================


🧠 Đang tải nội dung chi tiết từng job...


In [7]:
# --- Xuất ra CSV ---
# ===================================================
df = pd.DataFrame(all_jobs)
filename = f"job_data_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
df.to_csv(filename, index=False, encoding="utf-8-sig")

print("\n🎯 Crawl hoàn tất!")
print(f"📁 File lưu tại: {filename}")



🎯 Crawl hoàn tất!
📁 File lưu tại: job_data_20251028_150349.csv


In [24]:
import requests
import pandas as pd
from datetime import datetime

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) "
                  "Chrome/122.0.0.0 Safari/537.36",
    "Accept": "application/json, text/javascript, */*; q=0.01",
    "Referer": "https://www.topcv.vn/"
}

def fetch_topcv_jobs(keyword="", province="", max_pages=3):
    url = "https://www.topcv.vn/api/job-search"
    all_jobs = []

    for page in range(1, max_pages + 1):
        params = {"keyword": keyword, "page": page}
        if province:
            params["province"] = province

        resp = requests.get(url, headers=HEADERS, params=params, timeout=10)

        print(f"🔍 Trang {page} — Status {resp.status_code}")

        if resp.status_code != 200:
            print("❌ Lỗi khi tải trang:", resp.text[:200])
            break

        try:
            data = resp.json()
        except Exception as e:
            print("⚠️ Không parse được JSON:", e)
            print(resp.text[:500])
            break

        jobs = data.get("data", {}).get("jobs", [])
        if not jobs:
            print("⛔ Không còn việc ở trang này.")
            break

        all_jobs.extend(jobs)
    return all_jobs

if __name__ == "__main__":
    keyword = "data engineer"
    province = "ha-noi"

    jobs = fetch_topcv_jobs(keyword=keyword, province=province, max_pages=5)
    print(f"\n✅ Tổng số việc lấy được: {len(jobs)}")

    if jobs:
        df = pd.DataFrame(jobs)
        filename = f"topcv_jobs_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
        df.to_csv(filename, index=False, encoding="utf-8-sig")
        print(f"📁 Đã lưu file: {filename}")


🔍 Trang 1 — Status 404
❌ Lỗi khi tải trang: 
{"status":"failed","message":"404 Not found!"}

✅ Tổng số việc lấy được: 0


In [1]:
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from bs4 import BeautifulSoup
import pandas as pd
import time, random
from datetime import datetime

# ===================== CONFIG =====================
SITES = {
    "topcv": "https://www.topcv.vn/viec-lam",
    "careerviet": "https://www.careerlink.vn/vieclam/list",
    # Bạn có thể thử thêm các trang khác
}
MAX_SCROLL = 8
CRAWL_DATE = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
# ==================================================

# --- Khởi tạo Chrome ẩn danh thật ---
options = uc.ChromeOptions()
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("--start-maximized")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--disable-infobars")
options.add_argument("--disable-notifications")
options.add_argument("--lang=vi-VN,vi")

driver = uc.Chrome(options=options)

all_jobs = []

# --- Crawl từng trang ---
for site_name, start_url in SITES.items():
    print(f"\n💼 Đang crawl trang: {site_name}")
    driver.get(start_url)
    time.sleep(5 + random.uniform(1, 3))

    # --- Scroll xuống nhiều lần để load thêm việc ---
    for i in range(MAX_SCROLL):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(random.uniform(2.5, 4.0))

    # --- Lấy toàn bộ HTML sau khi load ---
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")

    # --- Lấy job blocks ---
    job_blocks = []
    if site_name == "topcv":
        job_blocks = soup.select("div.job-item-search-result") or soup.select("div.job-item")
    elif site_name == "careerviet":
        job_blocks = soup.select("div.job-link") or soup.select("div.job-item")

    print(f"🔍 Tìm thấy {len(job_blocks)} việc làm trên {site_name}")

    # --- Parse dữ liệu ---
    for job in job_blocks:
        try:
            title_elem = job.select_one("a[href]")
            if not title_elem:
                continue

            title = title_elem.get_text(strip=True)
            link = title_elem.get("href")
            if not link.startswith("http"):
                link = "https://" + site_name + ".vn" + link

            company = job.select_one(".company-name, .company a")
            location = job.select_one(".job-location, .address, .location")
            salary = job.select_one(".salary")
            posted_time = job.select_one(".time, .posted-date")

            job_data = {
                "source": site_name,
                "title": title,
                "company": company.get_text(strip=True) if company else None,
                "location": location.get_text(strip=True) if location else None,
                "salary": salary.get_text(strip=True) if salary else None,
                "time_posted": posted_time.get_text(strip=True) if posted_time else None,
                "link": link,
                "crawl_date": CRAWL_DATE,
            }
            all_jobs.append(job_data)
        except Exception as e:
            print("⚠️ Lỗi parse:", e)

print(f"\n✅ Tổng số việc thu được: {len(all_jobs)}")

driver.quit()

# --- Xuất dữ liệu ---
if all_jobs:
    df = pd.DataFrame(all_jobs)
    filename = f"jobs_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
    df.to_csv(filename, index=False, encoding="utf-8-sig")
    print(f"📁 Đã lưu file: {filename}")
else:
    print("⛔ Không lấy được việc nào, thử tăng MAX_SCROLL hoặc site khác.")


SessionNotCreatedException: Message: session not created: cannot connect to chrome at 127.0.0.1:50199
from session not created: This version of ChromeDriver only supports Chrome version 142
Current browser version is 141.0.7390.123; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#sessionnotcreatedexception
Stacktrace:
Symbols not available. Dumping unresolved backtrace:
	0x12c58c3
	0x12c5904
	0x10de76d
	0x111b20d
	0x111a159
	0x111008f
	0x110feb6
	0x1158993
	0x115830a
	0x114c766
	0x111dac0
	0x111ede4
	0x1547974
	0x1542bea
	0x12ee5b4
	0x12ddd28
	0x12e4d8d
	0x12cded8
	0x12ce09c
	0x12b7d1a
	0x76ce5d49
	0x7731d6db
	0x7731d661
